In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix, f1_score
from scipy import stats
import pandas as pd
import numpy as np
import pickle
import gc
import os

#garbage collector
import gc
gc.enable() #enabling collection

In [2]:
orderprdtrain_df = pd.read_csv(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\order_products__train.csv')
orderprdprior_df = pd.read_csv(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\order_products__prior.csv')
orders_df = pd.read_csv(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\orders.csv')
products_df = pd.read_csv(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\products.csv')
aisles_df = pd.read_csv(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\aisles.csv')
departments_df = pd.read_csv(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\departments.csv')

In [3]:
orderprdtrain_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [4]:
# merging order df, prior dataset, and products df

In [5]:
order_prior = pd.merge(orders_df, orderprdprior_df, on='order_id', how='inner')
order_prior = pd.merge(order_prior, products_df, on='product_id', how='left')
order_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17


In [6]:
#replace NaN with mean of 'days-since_prior_order'
mean = order_prior['days_since_prior_order'].mean()
mean = np.round(mean,2)
order_prior['days_since_prior_order'].fillna(mean, inplace=True)

In [7]:
order_prior.head(60)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2539329,1,prior,1,2,8,11.1,196,1,0,Soda,77,7
1,2539329,1,prior,1,2,8,11.1,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16
2,2539329,1,prior,1,2,8,11.1,12427,3,0,Original Beef Jerky,23,19
3,2539329,1,prior,1,2,8,11.1,26088,4,0,Aged White Cheddar Popcorn,23,19
4,2539329,1,prior,1,2,8,11.1,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17
5,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7
6,2398795,1,prior,2,3,7,15.0,10258,2,0,Pistachios,117,19
7,2398795,1,prior,2,3,7,15.0,12427,3,1,Original Beef Jerky,23,19
8,2398795,1,prior,2,3,7,15.0,13176,4,0,Bag of Organic Bananas,24,4
9,2398795,1,prior,2,3,7,15.0,26088,5,1,Aged White Cheddar Popcorn,23,19


# Creating features using user-ids
min, max and mean number of orders placed by each user

In [8]:
user_df = order_prior.groupby('user_id').agg({'order_number':(min, max, np.nanmean, 'count')}).round()
user_df.head()

order_number                  
                 min max nanmean count
user_id                               
1                  1  10     6.0    59
2                  1  14     8.0   195
3                  1  12     6.0    88
4                  1   5     3.0    18
5                  1   4     2.0    37

In [9]:
print('Null values -', user_df.isnull().any().sum())

Null values - 0


# Day of the week users ordered the most

In [10]:
# On which day of the the week did users order the most ?
user_df['max_day_of_week'] = order_prior.groupby(by=['user_id'])['order_dow'].\
aggregate(lambda x:x.value_counts().index[0])
user_df.head()

order_number                   max_day_of_week
                 min max nanmean count                
user_id                                               
1                  1  10     6.0    59               4
2                  1  14     8.0   195               2
3                  1  12     6.0    88               0
4                  1   5     3.0    18               4
5                  1   4     2.0    37               3

# Order hour of the day the users orders the most.

In [11]:
user_df['max_order_hour_of_day'] = order_prior.groupby(by=['user_id'])['order_dow'].\
aggregate(lambda x: stats.mode(x)[0])
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day
                 min max nanmean count                                      
user_id                                                                     
1                  1  10     6.0    59               4                     4
2                  1  14     8.0   195               2                     2
3                  1  12     6.0    88               0                     0
4                  1   5     3.0    18               4                     4
5                  1   4     2.0    37               3                     3

# User sum days since prior order

In [12]:
user_df['sum_days_since_prior_order'] = order_prior.groupby(by=['user_id'])\
['days_since_prior_order'].aggregate('sum')
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   
6                  1   3     2.0    14               4                     4   
7                  1  20    10.0   206               0                     0   
8                  1   3     2.0    49               6                     6   
9                  1   3     2.0    76               5                     5   
10                 1   5     3.0   143               5                     5   
11                 1   7     4.0    94               5                     5   
12                 1   5     4.0    74               5                     5   
13                 1  12     7.0    81               6                     6   
14                 1  13     7.0   210               5                     5   
15                 1  22    11.0    72               1                     1   
16                 1   6     3.0    70               2                     2   
17                 1  40    21.0   294               5                     5   
18                 1   6     3.0    39               2                     2   
19                 1   9     5.0   204               5                     5   
20                 1   4     3.0    22               2                     2   
21                 1  33    17.0   205               1                     1   
22                 1  15     8.0    69               5                     5   
23                 1   4     3.0    40               5                     5   
24                 1  18    10.0    38               0                     0   
25                 1   3     2.0    21               1                     1   
26                 1  12     8.0    56               3                     3   
27                 1  81    46.0   768               1                     1   
28                 1  24    13.0   183               0                     0   
29                 1  18    10.0   225               5                     5   
30                 1   8     4.0    11               1                     1   
31                 1  20    12.0   299               0                     0   
32                 1   5     3.0   114               3                     3   
33                 1   3     2.0    53               3                     3   
34                 1   5     4.0    32               4                     0   
35                 1   9     5.0   184               6                     6   
36                 1  37    15.0   153               4                     0   
37                 1  23    11.0   188               5                     5   
38                 1  12     6.0   195               6                     6   
39                 1   7     4.0   141               6                     6   
40                 1   9     5.0   104               0                     0   
41                 1   5     3.0    65               0                     0   
42                 1  16     8.0   133               3                     3   
43                 1  11     7.0   147               4                     4   
44                 1   3     2.0    37               0                     0   
45                 1   4     3.0    18               2                     2   
46                 1  19    10.0   157               0                     0   
47                 1   5     3.0    26               2                     2   
48      

# Mean number of days since prior orders for each user

In [13]:
user_df['mean_days_since_prior_order'] = order_prior.groupby(by=['user_id']).agg({'days_since_prior_order':np.nanmean})
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  
                                                                
user_id                                                         
1                           1149.5                   19.483051  
2                           3050.3                   15.642564  
3                           1007.0                   11.443182  
4                            259.4                   14.411111  
5                            499.1                   13.489189

# User Reordered Ratio

In [14]:
# User's reordered ratio
user_df['reordered_ratio'] = order_prior.groupby(by=['user_id']).agg({'reordered':np.nanmean})
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order reordered_ratio  
                                                                                
user_id                                                                         
1                           1149.5                   19.483051        0.694915  
2                           3050.3                   15.642564        0.476923  
3                           1007.0                   11.443182        0.625000  
4                            259.4                   14.411111        0.055556  
5                            499.1                   13.489189        0.378378

# User total number of orders placed

In [15]:
# total number of orders placed by a user-refers to each item
user_df['total_num_orders'] = order_prior.groupby(['user_id'])['order_number'].aggregate("sum")
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders  
                                          
user_id                                   
1              0.694915              343  
2              0.476923             1567  
3              0.625000              528  
4              0.055556               52  
5              0.378378               92

# User total products

In [16]:
# Total number of products brought by a user
user_df['user_total_prods'] =  order_prior.groupby(['user_id'])['product_id'].count()
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods  
                                                           
user_id                                                    
1              0.694915              343               59  
2              0.476923             1567              195  
3              0.625000              528               88  
4              0.055556               52               18  
5              0.378378               92               37

# User distinct products

In [17]:
# What are the distinct products brought by a User ?
user_df['user_unique_prods'] =  order_prior.groupby(['user_id'])['product_id'].aggregate(lambda x: x.nunique())
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  
                                                                             
user_id                                                                      
1              0.694915              343               59                18  
2              0.476923             1567              195               102  
3              0.625000              528               88                33  
4              0.055556               52               18                17  
5              0.378378               92               37                23

# User average basket

In [18]:
# User's average basket size
user_df['user_average_basket'] = user_df['user_total_prods']/user_df['total_num_orders']
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  \
                                                                              
user_id                                                                       
1              0.694915              343               59                18   
2              0.476923             1567              195               102   
3              0.625000              528               88                33   
4              0.055556               52               18                17   
5              0.378378               92               37                23   

        user_average_basket  
                             
user_id                      
1                  0.172012  
2                  0.124442  
3                  0.166667  
4                  0.346154  
5                  0.402174

# User order starts at

In [19]:
# At what number does the user's order number starts at ? This tells us if the user is an old customer or a new customer 
# of instacart
user_df['user_order_starts_at'] =  order_prior.groupby(['user_id'])['order_id'].aggregate(min)
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  \
                                                                              
user_id                                                                       
1              0.694915              343               59                18   
2              0.476923             1567              195               102   
3              0.625000              528               88                33   
4              0.055556               52               18                17   
5              0.378378               92               37                23   

        user_average_basket user_order_starts_at  
                                                  
user_id                                           
1                  0.172012               431534  
2                  0.124442               738281  
3                  0.166667               444309  
4                  0.346154                94891  
5                  0.402174               157374

# Average number of products brought by user in each purchase

In [20]:
# Lets first get the total products per order
total_prds_per_order =  order_prior.groupby(['user_id','order_id'])['product_id'].aggregate('count').\
to_frame('total_prds_per_order').reset_index()
total_prds_per_order.head()

,user_id,order_id,total_prds_per_order
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,2254736,5
4,1,2295261,6


In [21]:
# Average number of products brought by user in each purchase
user_df['avg_no_prds_each_purchase'] = total_prds_per_order.groupby(['user_id'])['total_prds_per_order'].mean()
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  \
                                                                              
user_id                                                                       
1              0.694915              343               59                18   
2              0.476923             1567              195               102   
3              0.625000              528               88                33   
4              0.055556               52               18                17   
5              0.378378               92               37                23   

        user_average_basket user_order_starts_at avg_no_prds_each_purchase  
                                                                            
user_id                                                                     
1                  0.172012               431534                  5.900000  
2                  0.124442               738281                 13.928571  
3                  0.166667               444309                  7.333333  
4                  0.346154                94891                  3.600000  
5                  0.402174               157374                  9.250000

In [22]:
# Median number of products brought by user in each purchase
user_df['median_no_prds_each_purchase'] = total_prds_per_order.groupby(['user_id'])['total_prds_per_order'].median()
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  \
                                                                              
user_id                                                                       
1              0.694915              343               59                18   
2              0.476923             1567              195               102   
3              0.625000              528               88                33   
4              0.055556               52               18                17   
5              0.378378               92               37                23   

        user_average_basket user_order_starts_at avg_no_prds_each_purchase  \
                                                                             
user_id                                                                      
1                  0.172012               431534                  5.900000   
2                  0.124442               738281                 13.928571   
3                  0.166667               444309                  7.333333   
4                  0.346154                94891                  3.600000   
5                  0.402174               157374                  9.250000   

        median_no_prds_each_purchase  
                                      
user_id                               
1                                5.5  
2                               13.5  
3                                7.0  
4                                3.0  
5                               10.0

In [23]:
# Minimum number of products brought by user in each purchase
user_df['min_no_prds_each_purchase'] = total_prds_per_order.groupby(['user_id'])['total_prds_per_order'].min()
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  \
                                                                              
user_id                                                                       
1              0.694915              343               59                18   
2              0.476923             1567              195               102   
3              0.625000              528               88                33   
4              0.055556               52               18                17   
5              0.378378               92               37                23   

        user_average_basket user_order_starts_at avg_no_prds_each_purchase  \
                                                                             
user_id                                                                      
1                  0.172012               431534                  5.900000   
2                  0.124442               738281                 13.928571   
3                  0.166667               444309                  7.333333   
4                  0.346154                94891                  3.600000   
5                  0.402174               157374                  9.250000   

        median_no_prds_each_purchase min_no_prds_each_purchase  
                                                                
user_id                                                         
1                                5.5                         4  
2                               13.5                         5  
3                                7.0                         5  
4                                3.0                         2  
5                               10.0                         5

In [24]:
# Maximum number of products brought by user in each purchase
user_df['max_no_prds_each_purchase'] = total_prds_per_order.groupby(['user_id'])['total_prds_per_order'].max()
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  \
                                                                              
user_id                                                                       
1              0.694915              343               59                18   
2              0.476923             1567              195               102   
3              0.625000              528               88                33   
4              0.055556               52               18                17   
5              0.378378               92               37                23   

        user_average_basket user_order_starts_at avg_no_prds_each_purchase  \
                                                                             
user_id                                                                      
1                  0.172012               431534                  5.900000   
2                  0.124442               738281                 13.928571   
3                  0.166667               444309                  7.333333   
4                  0.346154                94891                  3.600000   
5                  0.402174               157374                  9.250000   

        median_no_prds_each_purchase min_no_prds_each_purchase  \
                                                                 
user_id                                                          
1                                5.5                         4   
2                               13.5                         5   
3                                7.0                         5   
4                                3.0                         2   
5                               10.0                         5   

        max_no_prds_each_purchase  
                                   
user_id                            
1                               9  
2                              26  
3                              11  
4                               7  
5                              12

# Creating features using product-ids

# Number of times each product was purchased

In [25]:
# Total number of times each of the indivisual products were purchased
prd_df = pd.DataFrame()
prd_df['times_purchased'] = order_prior.groupby('product_id')['order_number'].count()
prd_df.head()

,times_purchased
product_id,
1,1852
2,90
3,277
4,329
5,15


# Total number of times the product was reordered - indicating how much the product was liked

In [26]:
# How many times was each product reordered ? Total number of times each product was reordered
prd_df['prod_tot_reorders'] = order_prior.groupby('product_id')['reordered'].aggregate('sum')
prd_df.head()

,times_purchased,prod_tot_reorders
product_id,,
1,1852,1136
2,90,12
3,277,203
4,329,147
5,15,9


# Product Reorder Ratio

In [27]:
# Product's reorder ratio
prd_df['prod_reordered_ratio'] = order_prior.groupby(['product_id'])['reordered'].aggregate("mean")
prd_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio
product_id,,,
1,1852,1136,0.613391
2,90,12,0.133333
3,277,203,0.732852
4,329,147,0.446809
5,15,9,0.600000


# Product average cart position(mean, median, max , min , sum, std)

In [28]:
# Statistic features of product's add to cart position
prd_df['avg_cart_position'] = order_prior.groupby(['product_id'])['add_to_cart_order'].aggregate("mean")
prd_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position
product_id,,,,
1,1852,1136,0.613391,5.801836
2,90,12,0.133333,9.888889
3,277,203,0.732852,6.415162
4,329,147,0.446809,9.507599
5,15,9,0.600000,6.466667


In [29]:
prd_df['median_cart_position'] = order_prior.groupby(['product_id'])['add_to_cart_order'].aggregate("median")
prd_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position
product_id,,,,,
1,1852,1136,0.613391,5.801836,4.0
2,90,12,0.133333,9.888889,8.0
3,277,203,0.732852,6.415162,4.0
4,329,147,0.446809,9.507599,8.0
5,15,9,0.600000,6.466667,6.0


In [30]:
prd_df['sum_cart_position'] = order_prior.groupby(['product_id'])['add_to_cart_order'].aggregate("sum")
prd_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position
product_id,,,,,,
1,1852,1136,0.613391,5.801836,4.0,10745
2,90,12,0.133333,9.888889,8.0,890
3,277,203,0.732852,6.415162,4.0,1777
4,329,147,0.446809,9.507599,8.0,3128
5,15,9,0.600000,6.466667,6.0,97


In [31]:
prd_df['min_cart_position'] = order_prior.groupby(['product_id'])['add_to_cart_order'].aggregate("min")
prd_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position
product_id,,,,,,,
1,1852,1136,0.613391,5.801836,4.0,10745,1
2,90,12,0.133333,9.888889,8.0,890,1
3,277,203,0.732852,6.415162,4.0,1777,1
4,329,147,0.446809,9.507599,8.0,3128,1
5,15,9,0.600000,6.466667,6.0,97,3


In [32]:
prd_df['max_cart_position'] = order_prior.groupby(['product_id'])['add_to_cart_order'].aggregate("max")
prd_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position
product_id,,,,,,,,
1,1852,1136,0.613391,5.801836,4.0,10745,1,74
2,90,12,0.133333,9.888889,8.0,890,1,45
3,277,203,0.732852,6.415162,4.0,1777,1,41
4,329,147,0.446809,9.507599,8.0,3128,1,36
5,15,9,0.600000,6.466667,6.0,97,3,16


In [33]:
prd_df['std_cart_position'] = order_prior.groupby(['product_id'])['add_to_cart_order'].aggregate("std")
prd_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position
product_id,,,,,,,,,
1,1852,1136,0.613391,5.801836,4.0,10745,1,74,5.575389
2,90,12,0.133333,9.888889,8.0,890,1,45,7.821670
3,277,203,0.732852,6.415162,4.0,1777,1,41,6.472701
4,329,147,0.446809,9.507599,8.0,3128,1,36,6.861485
5,15,9,0.600000,6.466667,6.0,97,3,16,3.563038


# Product's aisle and department reordered ratio

In [34]:
# Product's aisle reordered ratio
prod_aisle_df =  order_prior.groupby(['aisle_id'])['reordered'].aggregate("mean").to_frame('aisle_reorder_ratio')
prod_aisle_df = prod_aisle_df.reset_index()
prod_aisle_df.head()

,aisle_id,aisle_reorder_ratio
0,1,0.596597
1,2,0.489326
2,3,0.598007
3,4,0.489533
4,5,0.280627


In [35]:
# Product's department reordered ratio
prod_depart_df =  order_prior.groupby(['department_id'])['reordered'].aggregate("mean").to_frame('department_reorder_ratio')
prod_depart_df = prod_depart_df.reset_index()
prod_depart_df.head()

,department_id,department_reorder_ratio
0,1,0.541885
1,2,0.407980
2,3,0.628141
3,4,0.649913
4,5,0.569924


In [36]:
# Merging data
merge_data = pd.merge(products_df,prod_aisle_df,on='aisle_id')
merge_data = pd.merge(merge_data,prod_depart_df,on='department_id')
merge_data.head()

,product_id,product_name,aisle_id,department_id,aisle_reorder_ratio,department_reorder_ratio
0,1,Chocolate Sandwich Cookies,61,19,0.548698,0.57418
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,0.548698,0.57418
2,102,Danish Butter Cookies,61,19,0.548698,0.57418
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,0.548698,0.57418
4,285,Mini Nilla Wafers Munch Pack,61,19,0.548698,0.57418


In [37]:
merge_data = merge_data.drop(['product_name','aisle_id','department_id'],axis=1)
prd_df = pd.merge(prd_df,merge_data,on='product_id')
prd_df.head()

,product_id,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio
0,1,1852,1136,0.613391,5.801836,4.0,10745,1,74,5.575389,0.548698,0.574180
1,2,90,12,0.133333,9.888889,8.0,890,1,45,7.821670,0.152391,0.346721
2,3,277,203,0.732852,6.415162,4.0,1777,1,41,6.472701,0.527615,0.653460
3,4,329,147,0.446809,9.507599,8.0,3128,1,36,6.861485,0.556655,0.541885
4,5,15,9,0.600000,6.466667,6.0,97,3,16,3.563038,0.280627,0.346721


In [38]:
del merge_data,prod_depart_df,prod_aisle_df

# Creating features using user-ids and product-ids

In [39]:
# Number of times User 'A' bought product 'B'
user_x_product = pd.DataFrame()
user_x_product['times_brought'] = order_prior.groupby(by=['user_id','product_id'])['order_id'].aggregate('count')

In [40]:
user_x_product = user_x_product.reset_index()
user_x_product.head()

,user_id,product_id,times_brought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


# Number of times User 'A' reordered product 'B'

In [41]:
# How many times did User 'A' reorder product 'B' ?
reordered_count = order_prior.groupby(by=['user_id','product_id'])['reordered'].\
aggregate('sum').to_frame('uxp_total_reordered')

In [42]:
reordered_count = reordered_count.reset_index()
reordered_count.head()

,user_id,product_id,uxp_total_reordered
0,1,196,9
1,1,10258,8
2,1,10326,0
3,1,12427,9
4,1,13032,2


In [43]:
user_x_product = user_x_product.merge(reordered_count,on=['user_id','product_id'])
del reordered_count

# Finding when the user has bought a product the first time

In [44]:
# For which order number did a user buy each product ?
first_order_num = order_prior.groupby(by=['user_id','product_id'])['order_number'].\
aggregate('min').to_frame('uxp_first_order_num')

In [45]:
first_order_num = first_order_num.reset_index()
first_order_num.head()

,user_id,product_id,uxp_first_order_num
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [46]:
user_x_product = user_x_product.merge(first_order_num,on=['user_id','product_id'])
del first_order_num

# Average cart position of a product for each user

In [47]:
# Average add to cart position of a user for each product brought
avg_cart_pos_uxp = order_prior.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('mean').to_frame('uxp_avg_cart_pos')
avg_cart_pos_uxp = avg_cart_pos_uxp.reset_index()
avg_cart_pos_uxp.head()

,user_id,product_id,uxp_avg_cart_pos
0,1,196,1.400000
1,1,10258,3.333333
2,1,10326,5.000000
3,1,12427,3.300000
4,1,13032,6.333333


In [48]:
user_x_product = user_x_product.merge(avg_cart_pos_uxp,on=['user_id','product_id'])
del avg_cart_pos_uxp

# Minimum cart position of a product for each user

In [49]:
# Minimum add to cart position of a user for each product brought
min_cart_pos_uxp = order_prior.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('min').to_frame('uxp_min_cart_pos')
min_cart_pos_uxp = min_cart_pos_uxp.reset_index()
min_cart_pos_uxp.head()

,user_id,product_id,uxp_min_cart_pos
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,5


In [50]:
user_x_product = user_x_product.merge(min_cart_pos_uxp,on=['user_id','product_id'])
del min_cart_pos_uxp

# Maximum cart position of a product for each user

In [51]:
# Maximum add to cart position of a user for each product brought
max_cart_pos_uxp = order_prior.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('max').to_frame('uxp_max_cart_pos')
max_cart_pos_uxp = max_cart_pos_uxp.reset_index()
max_cart_pos_uxp.head()

,user_id,product_id,uxp_max_cart_pos
0,1,196,4
1,1,10258,6
2,1,10326,5
3,1,12427,9
4,1,13032,8


In [52]:
user_x_product = user_x_product.merge(max_cart_pos_uxp,on=['user_id','product_id'])
del max_cart_pos_uxp

# Median cart position of a product for each user

In [53]:
# Median add to cart position of a user for each product brought
median_cart_pos_uxp = order_prior.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('median').to_frame('uxp_median_cart_pos')
median_cart_pos_uxp = median_cart_pos_uxp.reset_index()
median_cart_pos_uxp.head()

,user_id,product_id,uxp_median_cart_pos
0,1,196,1.0
1,1,10258,3.0
2,1,10326,5.0
3,1,12427,2.5
4,1,13032,6.0


In [54]:
user_x_product = user_x_product.merge(median_cart_pos_uxp,on=['user_id','product_id'])
del median_cart_pos_uxp

In [55]:
user_x_product.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos
0,1,196,10,9,1,1.400000,1,4,1.0
1,1,10258,9,8,2,3.333333,2,6,3.0
2,1,10326,1,0,5,5.000000,5,5,5.0
3,1,12427,10,9,1,3.300000,1,9,2.5
4,1,13032,3,2,2,6.333333,5,8,6.0


# Finding when the user has bought a product the last time

In [56]:
last_order_num = order_prior.groupby(by=['user_id','product_id'])['order_number'].\
aggregate('max').to_frame('uxp_last_order_num')
last_order_num = last_order_num.reset_index()

In [57]:
last_order_num.head()

,user_id,product_id,uxp_last_order_num
0,1,196,10
1,1,10258,10
2,1,10326,5
3,1,12427,10
4,1,13032,10


In [58]:
user_x_product = user_x_product.merge(last_order_num,on=['user_id','product_id'])
del last_order_num

# Average,Median day of the week for purchased product

In [59]:
# Statistic features for day of the week for each purchased product
avg_dow = order_prior.groupby(by=['user_id','product_id'])['order_dow'].\
aggregate('mean').to_frame('uxp_avg_dow')
avg_dow = avg_dow.reset_index()

In [60]:
avg_dow.head()

,user_id,product_id,uxp_avg_dow
0,1,196,2.500000
1,1,10258,2.555556
2,1,10326,4.000000
3,1,12427,2.500000
4,1,13032,2.666667


In [61]:
median_dow = order_prior.groupby(by=['user_id','product_id'])['order_dow'].\
aggregate('median').to_frame('uxp_median_dow')
median_dow = median_dow.reset_index()

In [62]:
median_dow.head()

,user_id,product_id,uxp_median_dow
0,1,196,2.5
1,1,10258,3.0
2,1,10326,4.0
3,1,12427,2.5
4,1,13032,3.0


In [63]:
user_x_product = user_x_product.merge(avg_dow,on=['user_id','product_id'])
user_x_product = user_x_product.merge(median_dow,on=['user_id','product_id'])
del avg_dow,median_dow

# Average,Median hour of the day for purchased product

In [64]:
# statistic features for hour of the day for each purchased product
avg_hour_of_the_day = order_prior.groupby(by=['user_id','product_id'])['order_hour_of_day'].\
aggregate('mean').to_frame('uxp_avg_hour_of_the_day')
avg_hour_of_the_day = avg_hour_of_the_day.reset_index()

In [65]:
avg_hour_of_the_day.head()

,user_id,product_id,uxp_avg_hour_of_the_day
0,1,196,10.300000
1,1,10258,10.555556
2,1,10326,15.000000
3,1,12427,10.300000
4,1,13032,8.000000


In [66]:
median_hour_of_the_day = order_prior.groupby(by=['user_id','product_id'])['order_hour_of_day'].\
aggregate('median').to_frame('uxp_median_hour_of_the_day')
median_hour_of_the_day = median_hour_of_the_day.reset_index()

In [67]:
median_hour_of_the_day.head()

,user_id,product_id,uxp_median_hour_of_the_day
0,1,196,8.5
1,1,10258,9.0
2,1,10326,15.0
3,1,12427,8.5
4,1,13032,8.0


In [68]:
user_x_product = user_x_product.merge(avg_hour_of_the_day,on=['user_id','product_id'])
user_x_product = user_x_product.merge(median_hour_of_the_day,on=['user_id','product_id'])
del avg_hour_of_the_day,median_hour_of_the_day

In [69]:
# User-Product Order Rate - How frequent a product is brought by a user

In [70]:
#Total orders
total_orders = order_prior.groupby('user_id')['order_number'].max().\
to_frame('total_orders').reset_index()
total_orders.head()

,user_id,total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [71]:
total_orders_per_prod = order_prior.groupby(by=['user_id','product_id'])['order_number'].count().\
to_frame('total_orders_per_prod').reset_index()
total_orders_per_prod.head()

,user_id,product_id,total_orders_per_prod
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [72]:
#merging both data frames to calculate order rate for user-product
uxp_order_rate = pd.merge(total_orders,total_orders_per_prod,on='user_id')
uxp_order_rate.head()

,user_id,total_orders,product_id,total_orders_per_prod
0,1,10,196,10
1,1,10,10258,9
2,1,10,10326,1
3,1,10,12427,10
4,1,10,13032,3


In [73]:
uxp_order_rate['uxp_order_rate'] = uxp_order_rate['total_orders_per_prod'] / uxp_order_rate['total_orders']
uxp_order_rate.head()

,user_id,total_orders,product_id,total_orders_per_prod,uxp_order_rate
0,1,10,196,10,1.0
1,1,10,10258,9,0.9
2,1,10,10326,1,0.1
3,1,10,12427,10,1.0
4,1,10,13032,3,0.3


In [74]:
# drop total_orders and total orders per prod column to avoid collinearity between features
uxp_order_rate = uxp_order_rate.drop(['total_orders','total_orders_per_prod'],axis=1)
uxp_order_rate.head()

,user_id,product_id,uxp_order_rate
0,1,196,1.0
1,1,10258,0.9
2,1,10326,0.1
3,1,12427,1.0
4,1,13032,0.3


In [75]:
user_x_product = user_x_product.merge(uxp_order_rate,on=['user_id','product_id'])
del uxp_order_rate

# Sum across orders of the position in User A’s cart that Item B falls into.

In [76]:
# Sum across orders of the position in User A’s cart that Item B falls into.
sum_cart_pos_uxp = order_prior.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('sum').to_frame('uxp_sum_cart_pos')
sum_cart_pos_uxp = sum_cart_pos_uxp.reset_index()
sum_cart_pos_uxp.head()

,user_id,product_id,uxp_sum_cart_pos
0,1,196,14
1,1,10258,30
2,1,10326,5
3,1,12427,33
4,1,13032,19


In [77]:
user_x_product = user_x_product.merge(sum_cart_pos_uxp,on=['user_id','product_id'])
del sum_cart_pos_uxp

# Minimum user-item order days

In [78]:
# Mininum user-item order days
min_order_days_uxp = order_prior.groupby(by=['user_id','product_id'])['days_since_prior_order'].\
aggregate('min').to_frame('uxp_min_order_days')
min_order_days_uxp = min_order_days_uxp.reset_index()
min_order_days_uxp.head()

,user_id,product_id,uxp_min_order_days
0,1,196,0.0
1,1,10258,0.0
2,1,10326,28.0
3,1,12427,0.0
4,1,13032,15.0


In [79]:
user_x_product = user_x_product.merge(min_order_days_uxp,on=['user_id','product_id'])
del min_order_days_uxp

# Maximum user-item order days

In [80]:
# Maximum user-item order days
max_order_days_uxp = order_prior.groupby(by=['user_id','product_id'])['days_since_prior_order'].\
aggregate('max').to_frame('uxp_max_order_days')
max_order_days_uxp = max_order_days_uxp.reset_index()
max_order_days_uxp.head()

,user_id,product_id,uxp_max_order_days
0,1,196,30.0
1,1,10258,30.0
2,1,10326,28.0
3,1,12427,30.0
4,1,13032,30.0


# Median user-item order days

In [81]:
# Median user-item order days
median_order_days_uxp = order_prior.groupby(by=['user_id','product_id'])['days_since_prior_order'].\
aggregate('median').to_frame('uxp_median_order_days')
median_order_days_uxp = median_order_days_uxp.reset_index()
median_order_days_uxp.head()

,user_id,product_id,uxp_median_order_days
0,1,196,19.5
1,1,10258,20.0
2,1,10326,28.0
3,1,12427,19.5
4,1,13032,20.0


In [82]:
user_x_product = user_x_product.merge(max_order_days_uxp,on=['user_id','product_id'])
user_x_product = user_x_product.merge(median_order_days_uxp,on=['user_id','product_id'])
del median_order_days_uxp,max_order_days_uxp

# User Product Reorder Ratio

In [83]:
uxp_reorder_ratio = order_prior.groupby(by=['user_id','product_id'])['reordered'].aggregate("mean")\
.to_frame('uxp_reorder_ratio')
uxp_reorder_ratio = uxp_reorder_ratio.reset_index()
uxp_reorder_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,1,196,0.900000
1,1,10258,0.888889
2,1,10326,0.000000
3,1,12427,0.900000
4,1,13032,0.666667


In [93]:
user_x_product = user_x_product.merge(uxp_reorder_ratio,on=['user_id','product_id'])
del uxp_reorder_ratio

NameError: name 'uxp_reorder_ratio' is not defined

In [94]:
user_x_product.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,uxp_avg_dow,uxp_median_dow,uxp_avg_hour_of_the_day,uxp_median_hour_of_the_day,uxp_order_rate,uxp_sum_cart_pos,uxp_min_order_days,uxp_max_order_days,uxp_median_order_days,uxp_reorder_ratio
0,1,196,10,9,1,1.400000,1,4,1.0,10,2.500000,2.5,10.300000,8.5,1.0,14,0.0,30.0,19.5,0.900000
1,1,10258,9,8,2,3.333333,2,6,3.0,10,2.555556,3.0,10.555556,9.0,0.9,30,0.0,30.0,20.0,0.888889
2,1,10326,1,0,5,5.000000,5,5,5.0,5,4.000000,4.0,15.000000,15.0,0.1,5,28.0,28.0,28.0,0.000000
3,1,12427,10,9,1,3.300000,1,9,2.5,10,2.500000,2.5,10.300000,8.5,1.0,33,0.0,30.0,19.5,0.900000
4,1,13032,3,2,2,6.333333,5,8,6.0,10,2.666667,3.0,8.000000,8.0,0.3,19,15.0,30.0,20.0,0.666667


In [92]:
prd_df.head()

,product_id,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio
0,1,1852,1136,0.613391,5.801836,4.0,10745,1,74,5.575389,0.548698,0.574180
1,2,90,12,0.133333,9.888889,8.0,890,1,45,7.821670,0.152391,0.346721
2,3,277,203,0.732852,6.415162,4.0,1777,1,41,6.472701,0.527615,0.653460
3,4,329,147,0.446809,9.507599,8.0,3128,1,36,6.861485,0.556655,0.541885
4,5,15,9,0.600000,6.466667,6.0,97,3,16,3.563038,0.280627,0.346721


In [95]:
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  \
                                                                              
user_id                                                                       
1              0.694915              343               59                18   
2              0.476923             1567              195               102   
3              0.625000              528               88                33   
4              0.055556               52               18                17   
5              0.378378               92               37                23   

        user_average_basket user_order_starts_at avg_no_prds_each_purchase  \
                                                                             
user_id                                                                      
1                  0.172012               431534                  5.900000   
2                  0.124442               738281                 13.928571   
3                  0.166667               444309                  7.333333   
4                  0.346154                94891                  3.600000   
5                  0.402174               157374                  9.250000   

        median_no_prds_each_purchase min_no_prds_each_purchase  \
                                                                 
user_id                                                          
1                                5.5                         4   
2                               13.5                         5   
3                                7.0                         5   
4                                3.0                         2   
5                               10.0                         5   

        max_no_prds_each_purchase  
                                   
user_id                            
1                               9  
2                              26  
3                              11  
4                               7  
5                              12

In [99]:
user_df.to_pickle(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\user_df.pkl')
prd_df.to_pickle(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\prd_df.pkl')
user_x_product.to_pickle(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\user_x_product.pkl')